# Preparation


Install the necessary packages:


In [ ]:
! pip install numpy awkward vector uproot lz4 xxhash pandas matplotlib tqdm torch lightning torchmetrics


In [ ]:
import os
import numpy as np
import awkward as ak
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import vector
vector.register_awkward()


Here defines some helper functions to visualize a jet:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

typelist = ['ch+', 'ch-', 'nh', 'ph', 'el+', 'el-', 'mu+', 'mu-']


def make_subplot(ax, data, force_xylim=None):
    # default plotting configuration
    color_dict_ = {'ch': 'C0', 'nh': 'mediumpurple', 'ph': 'orange', 'el': 'red', 'mu': 'green'}
    color_dict = color_dict_.copy()
    color_dict.update({k + '+': color_dict_[k] for k in color_dict_})
    color_dict.update({k + '-': color_dict_[k] for k in color_dict_})
    if data.get('id') is None:
        data['id'] = ['default'] * len(data['pt'])
    if data.get('e') is None:
        for eta, phi, pt, id, d3d in zip(data['eta'], data['phi'], data['pt'], data['id'], data['d3d']):
            ptdraw = np.sqrt(pt) / 200
            alpha = 0.3
            if id in [4, 5]:
                ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 3, radius=ptdraw, clip_on=True,
                                                        alpha=alpha, edgecolor='black', **make_color_args(id, d3d)))
            elif id in [6, 7]:
                ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 3, radius=ptdraw, orientation=np.pi,
                                                        clip_on=True, alpha=alpha, edgecolor='black', **make_color_args(id, d3d)))
            elif id in [3]:
                ax.add_patch(mpl.patches.RegularPolygon((eta, phi), 5, radius=ptdraw,
                                                        clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
            else:
                ax.add_patch(plt.Circle((eta, phi), ptdraw, clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
    else:
        for eta, phi, pt, e, id, d3d in zip(data['eta'], data['phi'], data['pt'], data['e'], data['id'], data['d3d']):
            ax.add_patch(mpl.patches.Wedge((eta, phi), pt / 600., 90, 270,
                                           clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
            ax.add_patch(mpl.patches.Wedge((eta, phi), e / 600., 270, 90,
                                           clip_on=True, alpha=alpha, **make_color_args(id, d3d)))
    max_ang = force_xylim if force_xylim else max(max(abs(data['eta'])), max(abs(data['phi'])))
    # make square plot centered at (0,0)
    ax.set_xlim(-max_ang, max_ang)
    ax.set_ylim(-max_ang, max_ang)
    ax.set_xlabel(r'$\Delta\eta$')
    ax.set_ylabel(r'$\Delta\phi$')
    ax.set_aspect('equal')
    return max_ang


def make_color_args(id, d3d):
    color = color_fader('#74c476', '#081d58', d3d)
    if id in [2, 3]:
        return {'edgecolor': color, 'linewidth': 1, 'fill': False}
    else:
        return {'facecolor': color}


def color_fader(c1, c2, mix=0):  # fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    mix = min(1., mix)
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1 - mix) * c1 + mix * c2)


def visualize(arrays, idx=0, title=None, ax=None):
    data = {}
    data['pt'] = np.hypot(arrays[idx].part_px, arrays[idx].part_py)
    data['eta'] = arrays[idx].part_deta
    data['phi'] = arrays[idx].part_dphi
    data['d3d'] = np.tanh(np.hypot(arrays[idx].part_d0val, arrays[idx].part_dzval))
    part_type = np.concatenate([
        [(arrays[idx].part_isChargedHadron) & (arrays[idx].part_charge == 1)],
        [(arrays[idx].part_isChargedHadron) & (arrays[idx].part_charge == -1)],
        [arrays[idx].part_isNeutralHadron],
        [arrays[idx].part_isPhoton],
        [(arrays[idx].part_isElectron) & (arrays[idx].part_charge == 1)],
        [(arrays[idx].part_isElectron) & (arrays[idx].part_charge == -1)],
        [(arrays[idx].part_isMuon) & (arrays[idx].part_charge == 1)],
        [(arrays[idx].part_isMuon) & (arrays[idx].part_charge == -1)],
    ], axis=0)
    data['id'] = np.argmax(part_type.T, axis=1)  # better

    assert len(data['eta'] == data['id'])
    if ax is None:
        _, ax = plt.subplots(figsize=(5, 5))
    make_subplot(ax, data, force_xylim=0.5)
    if title:
        ax.set_title(title)
    return ax


# Download the dataset


In [ ]:
def download(url, fname, chunk_size=1024):
    '''https://gist.github.com/yanqd0/c13ed29e29432e3cf3e7c38467f42f51'''
    import requests
    from tqdm import tqdm

    if os.path.dirname(fname):
        os.makedirs(os.path.dirname(fname), exist_ok=True)

    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)


In [ ]:
signal_file = './JetClassMini/TTBar_000.root'
background_file = './JetClassMini/ZJetsToNuNu_000.root'

if not os.path.exists(signal_file):
    download('https://hqu.web.cern.ch/datasets/JetClassMini/TTBar_000.root', signal_file)
if not os.path.exists(background_file):
    download('https://hqu.web.cern.ch/datasets/JetClassMini/ZJetsToNuNu_000.root', background_file)


# Load the dataset


In [ ]:
jet_features = ['jet_pt', 'jet_eta', 'jet_phi', 'jet_energy', 'jet_nparticles',
                'jet_sdmass', 'jet_tau1', 'jet_tau2', 'jet_tau3', 'jet_tau4']
label_features = ['label_QCD', 'label_Tbqq']


In [ ]:
df_jet_signal = uproot.open(signal_file)['tree'].arrays(filter_name=jet_features + label_features, library='pd')
df_jet_signal.head(10)


In [ ]:
df_jet_background = uproot.open(background_file)['tree'].arrays(filter_name=jet_features + label_features, library='pd')
df_jet_background.head(10)


# Dataset splitting: training / validation / testing


In [ ]:
print('Number of signal jets:', len(df_jet_signal))
print('Number of background jets:', len(df_jet_background))


In this study, we are going to use 70% of the data for training, 10% for validation, and 20% for testing.

In [ ]:
test_slice = slice(0, 20000)
val_slice = slice(20000, 30000)
train_slice = slice(30000, None)


In [ ]:
sig_train, sig_val, sig_test = df_jet_signal[train_slice], df_jet_signal[val_slice], df_jet_signal[test_slice]
bkg_train, bkg_val, bkg_test = df_jet_background[train_slice], df_jet_background[val_slice], df_jet_background[test_slice]


In [ ]:
df_train = pd.concat([sig_train, bkg_train])
df_val = pd.concat([sig_val, bkg_val])
df_test = pd.concat([sig_test, bkg_test])
print(f'Training: {len(df_train)}, validation: {len(df_val)}, testing: {len(df_test)}')


# Training a fully-connected neural network


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import lightning as L
from torchmetrics.functional import accuracy


In [ ]:
class MLPModule(L.LightningModule):
    def __init__(self, lr=1e-3):
        super().__init__()
        self.lr = lr
        self.nn = nn.Sequential(
            nn.BatchNorm1d(10),
            nn.Linear(10, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2))

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        return self.nn(x)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        logits = self.nn(x)
        loss = F.cross_entropy(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        probs = F.softmax(logits)
        acc = accuracy(probs, y, task='multiclass', num_classes=2)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)
        return probs

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer


In [ ]:
class JetDataset(data.Dataset):

    def __init__(self, df, jet_features=jet_features, label_features=label_features):
        self.x = torch.tensor(df[jet_features].values, dtype=torch.float32)
        self.y = torch.tensor(df[label_features].values, dtype=torch.int32).argmax(1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


In [ ]:
train_ds = JetDataset(df_train)
val_ds = JetDataset(df_val)
test_ds = JetDataset(df_test)


In [ ]:
batch_size = 128
num_epochs = 20
lr = 1e-2


In [ ]:
model = MLPModule()
trainer = L.Trainer(
    max_epochs=num_epochs,
    logger=True
)
trainer.fit(model,
            data.DataLoader(train_ds, batch_size=batch_size, shuffle=True),
            data.DataLoader(val_ds, batch_size=batch_size, shuffle=True))


In [ ]:
trainer.test(model, data.DataLoader(test_ds, batch_size=batch_size))


# Performance evaluation

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc
from scipy.interpolate import interp1d


In [ ]:
model.eval()

In [ ]:
# Truth label: Top jet=1, q/g jet=0
y_true = df_test['label_Tbqq']


In [ ]:
preds = []
for x in data.DataLoader(test_ds, batch_size=128):
    preds.append(model.evaluate(x).detach().cpu().numpy())

prediction_probs = np.concatenate(preds)
prediction_probs


In [ ]:
plt.figure(figsize=(6, 6))
plt.hist([prediction_probs[y_true == 1, 1], prediction_probs[y_true == 0, 1]],
         label=['Top', 'q/g'], bins=50, histtype='step', density=True)
plt.yscale('log')
plt.xlabel("Prediction probability")
plt.legend()


In [ ]:
# fpr = epsilon_B, tpr = epsilon_S
fpr, tpr, thresholds = roc_curve(y_true=y_true, y_score=prediction_probs[:, 1])
auc_test = auc(fpr, tpr)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f'XGBoost (AUC={auc_test:.4f})')
plt.plot([0, 1], [0, 1], ls="--", color="k")
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend()


In [ ]:
prediction_class = prediction_probs.argmax(1)
prediction_class


In [ ]:
acc = accuracy_score(df_test['label_Tbqq'], prediction_class)
print(f'Accuracy: {acc}')


In [ ]:
background_eff_fn = interp1d(tpr, fpr)
background_eff_at_50 = background_eff_fn(0.5)
print(f"Backround rejection at signal efficiency 50%: {1/background_eff_at_50:.0f}")


In [ ]:
def compute_metrics(y_true, probs):
    fpr, tpr, _ = roc_curve(y_true=y_true, y_score=probs[:, 1])
    auc_test = auc(fpr, tpr)
    acc_test = accuracy_score(y_true, probs.argmax(1))
    background_eff_fn = interp1d(tpr, fpr)
    background_eff_at_50 = background_eff_fn(0.5)

    print(f"Accuracy: {acc_test:.4f}")
    print(f"AUC: {auc_test:.4f}")
    print(f"Backround rejection at 50% signal efficiency: {1/background_eff_at_50:.0f}")
    return fpr, tpr


In [ ]:
compute_metrics(y_true, prediction_probs)

# Visualize the results

In [ ]:
signal_table = uproot.open(signal_file)['tree'].arrays()
background_table = uproot.open(background_file)['tree'].arrays()


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
for idx in range(10):
    prob = model(JetDataset(df_jet_signal)[idx:idx + 1][0]).softmax(1)[0, 1]
    visualize(signal_table, idx, title=f'Top quark jet {idx}, prob(Top)={prob:.4f}', ax=axes[idx % 2][idx // 2])


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
for idx in range(10):
    prob = model(JetDataset(df_jet_background)[idx:idx + 1][0]).softmax(1)[0, 1]
    visualize(background_table, idx, title=f'q/g jet {idx}, prob(Top)={prob:.4f}', ax=axes[idx % 2][idx // 2])
